In [12]:
import pyDOE
import pandas as pd
import numpy as np
import Utils 
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import Bounds
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import namedtuple
from matplotlib import cm
import cma
from scipy.optimize import minimize
from scipy.optimize import Bounds
import sys

## Helper Functions

In [13]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' Plot the Graph of the Function'''
def plot_the_graph(x,y,z,graph_title):
    fig = plt.figure(figsize=(12,5))
    ax = Axes3D(fig)
    surf = ax.plot_trisurf(x,y,z,cmap=cm.jet, linewidth=0.3,alpha=0.9)
    fig.colorbar(surf, shrink=10, aspect=3)
    ax.set_xlabel('X1')
    ax.set_ylabel('X2')
    ax.set_zlabel('Robust(X1,X2)')
    plt.title(graph_title)
    plt.show()
    fig.savefig(graph_title+'.pdf')

''' Original Function for Optimization '''
def ackley_2D(X):
    x1,x2 = X
    A = (-20 *  np.exp (-0.2 * np.sqrt(( (x1**2) + (x2**2) ) / (2) ) ) )
    B = np.exp (((np.cos(2 * np.pi * x1 ) ) + (np.cos(2 * np.pi * x2 ) ) ) / (2) )
    return A - B +20 +np.exp(1)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=2, samples=n_obs, criterion='m')
    X1, X2 = lhd[:,0], lhd[:,1]
    return X1,X2


'''Robust Regularization based on minimax Principle 5% perturbations'''
def robust_regularization(X):
    x1,x2 = X
    eps = np.linspace(-6.5536,6.5536,2000)
    return np.max( ackley_2D([x1+eps,x2+eps]) )

''' Robustness based on a composite function of Mean and STd '''
def composite_robustness(X):
    w=1
    x1,x2 = X
    np.random.seed(0)
    eps = np.random.normal(loc=0.0,scale=1.0922666666666667,size=10000)
    sample_points = ackley_2D([x1+eps,x2+eps])
    sample_mean = np.mean(sample_points)
    variance = np.square(sample_points-sample_mean)
    std = np.sqrt(np.mean(variance))
    std = np.sqrt(np.mean(variance))
    return sample_mean + w * std

''' Generate Training Data using LHD along side the Output of the Robust System'''
def generate_training_data(n_obs):
    X1,X2 = DOE(n_obs)
    X1 = linearscaletransform(X1,range_out=(-32.768,32.768))
    X2 = linearscaletransform(X2,range_out=(-32.768,32.768))
    f_evaluation = ackley_2D([X1,X2])
    f_original = np.zeros(X1.shape[0])
    minimax_original = np.zeros(X1.shape[0])
    composite_original = np.zeros(X1.shape[0])
    for i in range(X1.shape[0]):
        f_original[i] = ackley_2D([X1[i],X2[i]])
        minimax_original[i] = robust_regularization([X1[i],X2[i]])
        composite_original[i] = composite_robustness([X1[i],X2[i]])
    train = pd.DataFrame()
    train['X1'] = pd.Series(X1)
    train['X2'] = pd.Series(X2)
    train['Y = F(X1,X2)'] = pd.Series(f_original)
    train['Y = robust_regularization(X1,X2)'] = pd.Series(minimax_original)
    train['Y = composite(X1,X2)'] = pd.Series(composite_original)
    train.to_csv('Training_Data_Sets\\'+str(n_obs)+'Samples.csv')
    return train

''' Generate Test Data using LHD along side the Output of the Robust System'''
def generate_test_data(n_obs):
    test = pd.DataFrame()
    np.random.seed(0)
    lhd = pyDOE.lhs(n=2, samples=n_obs, criterion='m')
    X1 = linearscaletransform(lhd[:,0],range_out=(-32.768,32.768))
    X2 = linearscaletransform(lhd[:,1],range_out=(-32.768,32.768))
    test ['X1'] = pd.Series(X1)
    test ['X2'] = pd.Series(X2)
    true_minimax = np.zeros(test.shape[0])
    true_composite = np.zeros(test.shape[0])
    for i in range(test.shape[0]):
        true_minimax[i] = robust_regularization(test.iloc[i,:])
        true_composite[i] = composite_robustness(test.iloc[i,:])
    test ['True_Minimax'] = pd.Series(true_minimax)
    test ['True_Composite'] = pd.Series(true_composite)
    test.to_csv('Test_Data_Sets\\'+str(n_obs)+'Samples.csv')
    return test

''' Check if There are same examples in Test and Train Data Sets'''
def check_data_sets(train,test):
    C = train.isin(test)
    C = pd.DataFrame.sum(C,axis=1)
    ids = C.index[C>1]
    print ('Total Number of Input Observations present in Test Data : '+str(ids.shape[0]))


## CMA-ES

In [14]:
opt = {'seed': 0 , 'maxfevals': 1e6  }
rr = cma.fmin(robust_regularization, 2 * [0] , 16.4 , options=opt  )
rc = cma.fmin(composite_robustness, 2 * [0] , 16.4 , options=opt  )

(3_w,6)-aCMA-ES (mu_w=2.0,w_1=63%) in dimension 2 (seed=172920, Sun Mar 31 11:29:44 2019)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      6 2.060583492128721e+01 1.0e+00 1.22e+01  1e+01  1e+01 0:00.0
    2     12 2.029063245203407e+01 1.2e+00 1.23e+01  1e+01  1e+01 0:00.0
    3     18 1.906410684945379e+01 1.5e+00 1.02e+01  7e+00  9e+00 0:00.0
  100    600 1.603191488399802e+01 9.2e+00 5.41e-05  2e-08  3e-08 0:00.3
  167   1002 1.603191486189179e+01 5.4e+00 1.69e-06  7e-12  9e-12 0:00.6
termination on tolx=1e-11 (Sun Mar 31 11:29:46 2019)
final/bestever f-value = 1.603191e+01 1.603191e+01
incumbent solution: [0.29496754290068566, -0.29496754289468163]
std deviation: [6.832524668634485e-12, 9.116565438650256e-12]
(3_w,6)-aCMA-ES (mu_w=2.0,w_1=63%) in dimension 2 (seed=166936, Sun Mar 31 11:29:46 2019)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      6 1.958187366784770e+01 1.0e+00 1.28e+01  1e+01  1e+01 0:00.0
    2   

In [15]:
print ('Xmin for RR :'+str(rr[0]))
print ('Fmin for RR :'+str(rr[1]))
print ('Xmin for RC :'+str(rc[0]))
print ('Fmin for RC :'+str(rc[1]))

Xmin for RR :[ 0.29496754 -0.29496754]
Fmin for RR :16.031914861891703
Xmin for RC :[-3.80869461 -5.22576592]
Fmin for RC :15.301781731408404


## SLSQP

In [16]:
const = Bounds([-32.768, -32.768], [32.768, 32.768])
min_robust = minimize(robust_regularization,(0,0),method='SLSQP',bounds=const)
min_compo = minimize(composite_robustness,(0,0),method='SLSQP',bounds=const)

In [17]:
min_robust

     fun: 16.936628029225954
     jac: array([0.13527513, 0.13527513])
 message: 'Optimization terminated successfully.'
    nfev: 15
     nit: 1
    njev: 1
  status: 0
 success: True
       x: array([-8.7176233e-07, -8.7176233e-07])

In [18]:
min_compo

     fun: 6.753662007025669
     jac: array([-0.00060558, -0.00060558])
 message: 'Optimization terminated successfully.'
    nfev: 16
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([0.00969311, 0.00969311])

In [19]:
np.random.seed(0)
lhd = pyDOE.lhs(n=2, samples= 1000, criterion='m')
X1 = linearscaletransform(lhd[:,0],range_out=(-32.768,32.768))
X2 = linearscaletransform(lhd[:,1],range_out=(-32.768,32.768))
Data = pd.DataFrame()
Data['X1'] = pd.Series(X1)
Data['X2'] = pd.Series(X2)
Data.to_csv('Optimization_Test.csv')

## Meta-Modelling

In [20]:
train = pd.read_csv(sys.path[1]+str('\\Training_Data_Sets\\100Samples.csv')).iloc[:,1:]
test = pd.read_csv('Optimization_Test.csv').iloc[:,1:]
model_k_m,pred_k_m = Utils.kriging_minimax(train,test.iloc[:,:2])
model_k_c,pred_k_c = Utils.kriging_composite(train,test.iloc[:,:2])
model_r_m,pred_r_m = Utils.rf_minimax(train,test.iloc[:,:2])
model_r_c,pred_r_c = Utils.rf_composite(train,test.iloc[:,:2])
model_n_m,pred_n_m = Utils.KNN_minimax(train,test.iloc[:,:2])
model_n_c,pred_n_c = Utils.KNN_composite(train,test.iloc[:,:2])
model_s_m,pred_s_m = Utils.SVR_minimax(train,test.iloc[:,:2])
model_s_c,pred_s_c = Utils.SVR_composite(train,test.iloc[:,:2])
model_b_m,pred_b_m = Utils.RBF_minimax(train,test.iloc[:,:2])
model_b_c,pred_b_c = Utils.RBF_composite(train,test.iloc[:,:2])
train ['X1X2'] = pd.Series( np.array(train.X1) * np.array(train.X2) )
train ['X1**2'] = pd.Series ( np.array(train.X1)**2 )
train ['X2**2'] = pd.Series ( np.array(train.X2)**2 )
f_original = train['Y = F(X1,X2)']
robust_original = train['Y = robust_regularization(X1,X2)']
composite_original = train ['Y = composite(X1,X2)']
del train['Y = F(X1,X2)']
del train['Y = robust_regularization(X1,X2)']
del train ['Y = composite(X1,X2)']
train['Y = F(X1,X2)'] = f_original
train['Y = robust_regularization(X1,X2)'] = robust_original
train ['Y = composite(X1,X2)'] = composite_original
test ['X1X2'] =  pd.Series( np.array(test.X1) * np.array(test.X2) )
test ['X1**2'] = pd.Series ( np.array(test.X1)**2 )
test ['X2**2'] = pd.Series ( np.array(test.X2)**2 )
model_m_m,pred_m_m = Utils.elastic_net_minimax(train,test.iloc[:,:5])
model_m_c,pred_m_c = Utils.elastic_net_composite(train,test.iloc[:,:5])
del train['X1X2']
del train['X1**2']
del train['X2**2']
del test['X1X2']
del test['X1**2']
del test['X2**2']

C:\Users\sefi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [21]:
print ('RR :  Minimum Using Kriging is : '+str ( robust_regularization(np.array(test.iloc[np.argmin(pred_k_m),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_k_m),:])))
print ('RC :  Minimum Using Kriging is : '+str ( composite_robustness(np.array(test.iloc[np.argmin(pred_k_c),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_k_c),:])))

RR :  Minimum Using Kriging is : 17.014841059206834
At X = :[-1.56407075  0.51378406]
RC :  Minimum Using Kriging is : 7.728206585775058
At X = :[-1.20453403 -0.04415047]


In [22]:
print ('RR :  Minimum Using SVR  is : '+str ( robust_regularization(np.array(test.iloc[np.argmin(pred_s_m),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_s_m),:])))
print ('RC :  Minimum Using SVR is : '+str ( composite_robustness(np.array(test.iloc[np.argmin(pred_s_c),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_s_c),:])))

RR :  Minimum Using SVR  is : 17.014841059206834
At X = :[-1.56407075  0.51378406]
RC :  Minimum Using SVR is : 7.709349436319826
At X = :[-0.80030553 -0.71519228]


In [23]:
print ('RR :  Minimum Using KNN is : '+str ( robust_regularization(np.array(test.iloc[np.argmin(pred_n_m),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_n_m),:])))
print ('RC :  Minimum Using KNN is : '+str ( composite_robustness(np.array(test.iloc[np.argmin(pred_n_c),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_n_c),:])))

RR :  Minimum Using KNN is : 17.517316929006554
At X = :[-3.25609051  1.26503354]
RC :  Minimum Using KNN is : 9.208888928118673
At X = :[1.95129025 0.67959169]


In [24]:
print ('RR :  Minimum Using RF is : '+str ( robust_regularization(np.array(test.iloc[np.argmin(pred_r_m),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_r_m),:])))
print ('RC :  Minimum Using RF is : '+str ( composite_robustness(np.array(test.iloc[np.argmin(pred_r_c),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_r_c),:])))

RR :  Minimum Using RF is : 17.975237352254663
At X = :[-3.36653792  0.12298411]
RC :  Minimum Using RF is : 11.818671344789811
At X = :[2.72416023 2.19294009]


In [25]:
print ('RR :  Minimum Using RBFN is : '+str ( robust_regularization(np.array(test.iloc[np.argmin(pred_b_m),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_b_m),:])))
print ('RC :  Minimum Using RBFN is : '+str ( composite_robustness(np.array(test.iloc[np.argmin(pred_b_c),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_b_c),:])))

RR :  Minimum Using RBFN is : 22.051826676914587
At X = :[-32.33262943  32.01916015]
RC :  Minimum Using RBFN is : 7.728206585775058
At X = :[-1.20453403 -0.04415047]


In [26]:
print ('RR :  Minimum Using ELN is : '+str ( robust_regularization(np.array(test.iloc[np.argmin(pred_m_m),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_m_m),:])))
print ('RC :  Minimum Using ELN is : '+str ( composite_robustness(np.array(test.iloc[np.argmin(pred_m_c),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_m_c),:])))

RR :  Minimum Using ELN is : 17.38835785798418
At X = :[-0.80030553 -0.71519228]
RC :  Minimum Using ELN is : 7.709349436319826
At X = :[-0.80030553 -0.71519228]
